In [1]:
!pip install requests

In [2]:
import sqlite3
import requests
import logging
from datetime import datetime

API_KEY = "00a73f46b4d1d1944343492cfa581080"

DB_NAME = "weather_data.db"
CITIES = ["Mumbai", "Delhi", "Bangalore", "Chennai", "Kolkata"]

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

def get_connection():
    return sqlite3.connect(DB_NAME)

In [3]:
def setup_database():
    conn = get_connection()
    cursor = conn.cursor()

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS cities (
            city_id INTEGER PRIMARY KEY AUTOINCREMENT,
            city_name TEXT UNIQUE NOT NULL
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS weather_records (
            record_id INTEGER PRIMARY KEY AUTOINCREMENT,
            city_id INTEGER,
            recorded_at TIMESTAMP,
            temperature REAL,
            humidity INTEGER,
            pressure REAL,
            wind_speed REAL,
            condition TEXT,
            FOREIGN KEY(city_id) REFERENCES cities(city_id)
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS pipeline_logs (
            log_id INTEGER PRIMARY KEY AUTOINCREMENT,
            run_time TIMESTAMP,
            status TEXT,
            records_processed INTEGER,
            error_message TEXT
        )
    """)

    conn.commit()
    conn.close()

setup_database()

In [4]:
BASE_URL = "https://api.openweathermap.org/data/2.5/weather"

def fetch_weather(city_name):
    params = {
        "q": city_name,
        "appid": API_KEY,
        "units": "metric"
    }

    try:
        response = requests.get(BASE_URL, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()

        return {
            "city": city_name,
            "temperature": data["main"]["temp"],
            "humidity": data["main"]["humidity"],
            "pressure": data["main"]["pressure"],
            "wind_speed": data["wind"]["speed"],
            "condition": data["weather"][0]["description"],
            "timestamp": datetime.utcnow()
        }

    except Exception as e:
        logging.error(str(e))
        return None

In [5]:
def validate_weather(data):
    if data is None:
        return False
    if not (-100 <= data["temperature"] <= 70):
        return False
    if not (0 <= data["humidity"] <= 100):
        return False
    if not (800 <= data["pressure"] <= 1200):
        return False
    if data["wind_speed"] < 0:
        return False
    return True

In [6]:
def run_pipeline():
    conn = get_connection()
    cursor = conn.cursor()
    success_count = 0

    try:
        for city in CITIES:
            data = fetch_weather(city)

            if validate_weather(data):
                cursor.execute(
                    "INSERT OR IGNORE INTO cities (city_name) VALUES (?)",
                    (city,)
                )

                cursor.execute(
                    "SELECT city_id FROM cities WHERE city_name = ?",
                    (city,)
                )
                city_id = cursor.fetchone()[0]

                cursor.execute("""
                    INSERT INTO weather_records
                    (city_id, recorded_at, temperature, humidity, pressure, wind_speed, condition)
                    VALUES (?, ?, ?, ?, ?, ?, ?)
                """, (
                    city_id,
                    data["timestamp"],
                    data["temperature"],
                    data["humidity"],
                    data["pressure"],
                    data["wind_speed"],
                    data["condition"]
                ))

                success_count += 1

        conn.commit()

        cursor.execute("""
            INSERT INTO pipeline_logs (run_time, status, records_processed, error_message)
            VALUES (?, ?, ?, ?)
        """, (datetime.utcnow(), "SUCCESS", success_count, None))

        conn.commit()
        print(f"Pipeline completed: {success_count} records inserted")

    except Exception as e:
        conn.rollback()
        cursor.execute("""
            INSERT INTO pipeline_logs (run_time, status, records_processed, error_message)
            VALUES (?, ?, ?, ?)
        """, (datetime.utcnow(), "FAILED", success_count, str(e)))
        conn.commit()
        print("Pipeline failed")

    finally:
        conn.close()

run_pipeline()

/tmp/ipython-input-413264295.py:22: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow()
/tmp/ipython-input-3211431632.py:22: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute("""


Pipeline completed: 5 records inserted


/tmp/ipython-input-3211431632.py:43: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  """, (datetime.utcnow(), "SUCCESS", success_count, None))
/tmp/ipython-input-3211431632.py:40: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute("""


In [8]:
def highest_average_temperature():
    conn = get_connection()
    cursor = conn.cursor()

    cursor.execute("""
        SELECT c.city_name, ROUND(AVG(w.temperature), 2) AS avg_temp
        FROM weather_records w
        JOIN cities c ON w.city_id = c.city_id
        GROUP BY c.city_name
        ORDER BY avg_temp DESC
    """)

    results = cursor.fetchall()
    conn.close()

    if not results:
        print("No weather data available.")
        return []

    print("\nAverage Temperature by City (°C)\n")
    print("-" * 35)

    for city, avg_temp in results:
        print(f"{city:<12} | {avg_temp:.2f}°C")

    return results

In [9]:
highest_average_temperature()


Average Temperature by City (°C)

-----------------------------------
Mumbai       | 27.99°C
Chennai      | 27.47°C
Bangalore    | 26.42°C
Kolkata      | 23.97°C
Delhi        | 23.05°C


[('Mumbai', 27.99),
 ('Chennai', 27.47),
 ('Bangalore', 26.42),
 ('Kolkata', 23.97),
 ('Delhi', 23.05)]